<a href="https://colab.research.google.com/github/coldsober-irene/dipulearning/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [29]:
import os, cv2, shutil
from matplotlib import pyplot as plt
import random, pickle, numpy as np

In [60]:
dir = '/content/drive/MyDrive/Dataset'
categories = [category for category in os.listdir(dir)]
imsize = 220
print("CATEGORIES: ", categories)
images_needed = []
for category in categories:
  cat_path = os.path.join(dir, category)
  class_ = categories.index(category)
  for im in os.listdir(cat_path):
    image = os.path.join(cat_path, im)
    im_array = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    same_size = cv2.resize(im_array, (imsize, imsize))
    images_needed.append([same_size, class_])
    # plt.imshow(same_size, cmap = 'gray')
    # plt.show()

random.shuffle(images_needed)
train_data = np.array([row[0] for row in images_needed]).reshape(-1, imsize, imsize, 1)
labels = np.array([row[1] for row in images_needed])



CATEGORIES:  ['Motorcycle', 'Car', 'Auto', '.ipynb_checkpoints']


In [61]:

with open(os.path.join(dir,"train.pickle"), 'wb') as f:
  pickle.dump(train_data, f)

with open(os.path.join(dir,"labels.pickle"), 'wb') as f:
  pickle.dump(labels, f)

In [62]:

train = pickle.load(open(os.path.join(dir,"train.pickle"), 'rb'))
lab = pickle.load(open(os.path.join(dir,"labels.pickle"), 'rb'))

train = np.array(train)
train = train / 255


In [66]:
train.shape, lab.shape

((1326, 220, 220, 1), (1326,))

In [70]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Flatten
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical


#(batch_size, height, width, channels)

In [71]:
model = Sequential()
# LAYER 1
model.add(Conv2D(64, (3, 3), input_shape = train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
# LAYER 2
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
# LAYER 3
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
# LAYER 4
model.add(Flatten())
model.add(Dense(128))

# LAYER 5
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
lab = to_categorical(lab)
model.fit(train, lab,batch_size=32, validation_split=0.2)

38/38 [==============================] - 316s 8s/step - loss: 1.2240 - accuracy: 0.5901 - val_loss: 0.4982 - val_accuracy: 0.7744
